In [2]:
import gradio as gr
from PIL import Image
import os

gr.close_all()

# Your image path
nano_img = "/mnt/data/ChatGPT Image Nov 29, 2025, 05_00_07 PM.png"

# ----- SAVE EDITED IMAGE -----
def save_edits(edited_img):
    return edited_img


# ----- ZOOM VIEWER HTML -----
def zoom_html():
    return f"""
    <style>
    #zoom_box {{
        width: 100%;
        height: 600px;
        border: 2px solid #333;
        border-radius: 10px;
    }}
    </style>

    <div id="zoom_box"></div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/openseadragon/4.1.0/openseadragon.min.js"></script>

    <script>
    var viewer = OpenSeadragon({{
        id: "zoom_box",
        prefixUrl: "https://cdnjs.cloudflare.com/ajax/libs/openseadragon/4.1.0/images/",
        tileSources: {{
            type: 'image',
            url: 'file://{nano_img}'
        }},
        showNavigator: true,
        navigatorPosition: "BOTTOM_RIGHT",
    }});
    </script>
    """


# ---------- UI ----------

with gr.Blocks(title="Nano-chip Annotator + Zoom Viewer") as demo:

    gr.Markdown("## 🔬 **Nano-Chip Annotator + High-Resolution Zoom Viewer**")

    with gr.Tab("✏️ Annotate / Draw"):
        gr.Markdown("Use drawing tools to mark chip areas, circuits, labels, etc.")

        editor = gr.ImageEditor(
            value=nano_img,
            label="Draw / Box / Text on the nano-chip image",
            height=500
        )

        save_btn = gr.Button("💾 Save Edited Image")
        saved_output = gr.Image(label="Final Annotated Image")

        save_btn.click(save_edits, inputs=editor, outputs=saved_output)


    with gr.Tab("🔍 High-Res Zoom Viewer"):
        gr.Markdown("Ultra zoom into microchip details:")
        gr.HTML(zoom_html())


demo.launch()



* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [4]:
# Nano-chip Annotator + Auto-label + Compare (Upload Required, No Default Image)
import os, io, json, cv2, numpy as np
from PIL import Image, ImageDraw, ImageFont
import gradio as gr
import subprocess, sys

gr.close_all()

# ----- try to ensure cv2 is installed -----
try:
    import cv2
except:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "opencv-python"])
    import cv2


# ----- Draw bounding boxes -----
def draw_boxes_on_pil(pil_img, boxes, labels):
    img = pil_img.convert("RGBA")
    overlay = Image.new("RGBA", img.size, (0,0,0,0))
    draw = ImageDraw.Draw(overlay)

    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 18)
    except:
        font = ImageFont.load_default()

    for (x,y,w,h), lab in zip(boxes, labels):
        x1, y1, x2, y2 = x, y, x+w, y+h
        draw.rectangle([x1,y1,x2,y2], outline=(255,165,0), width=3)
        tw, th = draw.textsize(lab, font)
        draw.rectangle([x1, y1-th-4, x1+tw+6, y1], fill=(0,0,0,150))
        draw.text((x1+3,y1-th-2), lab, font=font, fill=(255,255,255))

    final = Image.alpha_composite(img, overlay).convert("RGB")
    return final


# ----- Heuristic detector for chip traces -----
def heuristic_detect_components(cv_img):
    gray = cv2.cvtColor(cv_img, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    th = cv2.adaptiveThreshold(
        blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    closed = cv2.morphologyEx(th, cv2.MORPH_CLOSE, kernel, iterations=2)

    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    h, w = gray.shape
    boxes, labels = [], []

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < (h*w)*0.00005:
            continue

        x,y,wc,hc = cv2.boundingRect(cnt)
        ar = wc/float(hc)

        if area < (h*w)*0.0008 and 0.8 <= ar <= 1.2:
            lab = "pad"
        elif ar > 4 or ar < 0.25:
            lab = "trace"
        elif area > (h*w)*0.01:
            lab = "component"
        else:
            perimeter = cv2.arcLength(cnt, True)
            if perimeter > 0:
                circularity = 4*np.pi*(area/(perimeter*perimeter))
            else:
                circularity = 0

            lab = "via" if circularity > 0.4 else "component"

        boxes.append((x,y,wc,hc))
        labels.append(lab)

    return boxes, labels


# ----- Main Auto-label + Compare -----
def run_auto_label(upload_img, edited_img):
    if upload_img is None:
        return None, None, None, "Please upload an image first!"

    # Convert upload to cv format
    pil = Image.open(upload_img.name).convert("RGB")
    cv_img = cv2.cvtColor(np.array(pil), cv2.COLOR_RGB2BGR)

    # Run heuristic labeling
    boxes, labels = heuristic_detect_components(cv_img)
    auto_pil = draw_boxes_on_pil(pil, boxes, labels)

    # Prepare edited image
    if isinstance(edited_img, np.ndarray):
        edited_pil = Image.fromarray(edited_img.astype("uint8"))
    else:
        edited_pil = pil.copy()

    # Side-by-side comparison image
    h = max(pil.height, auto_pil.height, edited_pil.height)
    def resize(img): return img.resize((int(img.width*(h/img.height)), h))

    o = resize(pil)
    a = resize(auto_pil)
    e = resize(edited_pil)

    final = Image.new("RGB", (o.width + a.width + e.width + 40, h), (30,30,30))
    final.paste(o, (10,0))
    final.paste(a, (o.width+20,0))
    final.paste(e, (o.width+a.width+30,0))

    # JSON annotations
    ann = {"annotations":[]}
    for (x,y,wc,hc), lab in zip(boxes, labels):
        ann["annotations"].append({
            "label": lab, "x":x, "y":y, "w":wc, "h":hc
        })

    return auto_pil, final, json.dumps(ann, indent=2), None


# ----- Build UI -----
with gr.Blocks(title="Nano-chip Auto-Label Annotator") as demo:

    gr.Markdown("## 🔬 Nano-Chip Auto-Label + Editor + Side-by-Side Compare")

    upload = gr.File(label="Upload Nano-chip Image (.png/.jpg)")
    editor = gr.ImageEditor(label="Draw / Edit before auto-label", height=450)

    run_btn = gr.Button("🪄 Auto-Label & Compare", variant="primary")

    auto_out = gr.Image(label="Auto-Labeled Image")
    sbs_out = gr.Image(label="Side-by-side (Original | Auto | Edited)")
    json_out = gr.Textbox(label="Annotations JSON", lines=12)

    msg_out = gr.Textbox(label="Messages")

    run_btn.click(
        fn=run_auto_label,
        inputs=[upload, editor],
        outputs=[auto_out, sbs_out, json_out, msg_out]
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
